In [37]:
import pandas as pd
from Transaction import *
from nacl.signing import SigningKey
from nacl.encoding import Base64Encoder
from hashlib import sha256
from nacl.encoding import HexEncoder

In [38]:
# create the keys first using a dataframe 
keys_list = pd.DataFrame()

In [39]:
# generate a list of 8 public keys and 8 private keys along with them and the signatures along with them since the signatures should stay the same for each public key
pub_keys_list = []
secret_keys_list = []
users_list = []
user = 'user'
for x in range(0, 8):
    users_list.append(user + str(x))
    sk = SigningKey.generate()
    secret_keys_list.append(sk)
    pk = sk.verify_key
    pub_keys_list.append(pk)

In [40]:
# add these two list to the dataframe
keys_list['users'] = users_list
keys_list['pubkey'] = pub_keys_list
keys_list['sk'] = secret_keys_list

In [41]:
def createGenesisSignature(output, user_sk):
    temp = output.encode()
    signature = user_sk.sign(temp, encoder=Base64Encoder).decode()
    return signature

In [42]:
# create the genesis transaction first - create 8 different ones 
output_1 = transaction_output("100", keys_list['pubkey'][0].encode(encoder=Base64Encoder).decode())
output_2 = transaction_output("100", keys_list['pubkey'][1].encode(encoder=Base64Encoder).decode())
output_3 = transaction_output("100", keys_list['pubkey'][2].encode(encoder=Base64Encoder).decode())
output_4 = transaction_output("100", keys_list['pubkey'][3].encode(encoder=Base64Encoder).decode())
output_5 = transaction_output("100", keys_list['pubkey'][4].encode(encoder=Base64Encoder).decode())
output_6 = transaction_output("100", keys_list['pubkey'][5].encode(encoder=Base64Encoder).decode())
output_7 = transaction_output("100", keys_list['pubkey'][6].encode(encoder=Base64Encoder).decode())
output_8 = transaction_output("100", keys_list['pubkey'][7].encode(encoder=Base64Encoder).decode())

In [43]:
all_outputs = []
all_outputs.append(output_1)
all_outputs.append(output_2)
all_outputs.append(output_3)
all_outputs.append(output_4)
all_outputs.append(output_5)
all_outputs.append(output_6)
all_outputs.append(output_7)
all_outputs.append(output_8)

In [44]:
g_sig = createGenesisSignature(json.dumps([o.to_dict() for o in all_outputs]), keys_list['sk'][0])
g_transaction = Transaction([], all_outputs, g_sig, None, True)
json_object = json.dumps(g_transaction.to_dict(), indent=4)
with open("transactions/genesis_tx.json", "w") as outfile:
    outfile.write(json_object)

# Create file for double spending 

In [45]:
def createSignature(input, output, user_sk):
    temp = input.encode()
    temp += output.encode()
    signature = user_sk.sign(temp, encoder=Base64Encoder).decode()
    return signature

In [46]:
ds_input = []
ds_output = []
ds_input_tx = transaction_input("1", output_1, None)
ds_output_tx1 = transaction_output("50", keys_list['pubkey'][0].encode(encoder=Base64Encoder).decode(), None)
ds_output_tx2 = transaction_output("50", keys_list['pubkey'][1].encode(encoder=Base64Encoder).decode(), None)
ds_input.append(ds_input_tx)
ds_output.append(ds_output_tx1)
ds_output.append(ds_output_tx2)
ds_sig = createSignature(json.dumps(([i.to_dict() for i in ds_input])),json.dumps([o.to_dict() for o in ds_output]), keys_list['sk'][0])
ds_1_Transaction = Transaction(ds_input, ds_output, ds_sig, None, False)

1st hash :005f2513d6d976698e7d9c6bc859a81ee288729404f5e1db6fa7842e9e40e880


2nd Transaction for Double Spend 

In [47]:
ds_total_input = []
ds_total_output = []
second_ds_input_tx = transaction_input("1", output_1, None)
second_ds_output_tx1 = transaction_output("50", keys_list['pubkey'][2].encode(encoder=Base64Encoder).decode(), None)
second_ds_output_tx2 = transaction_output("50", keys_list['pubkey'][3].encode(encoder=Base64Encoder).decode(), None)
ds_total_input.append(second_ds_input_tx)
ds_total_output.append(second_ds_output_tx1)
ds_total_output.append(second_ds_output_tx2)
ds_sig = createSignature(json.dumps(([i.to_dict() for i in ds_total_input])),json.dumps([o.to_dict() for o in ds_total_output]), keys_list['sk'][0])
ds_2_Transaction = Transaction(ds_total_input, ds_total_output, ds_sig, None, False)

1st hash :fb38e7c432eec7dd32f87422159bd1cff977b98b537a8738d32d0b24e6d0c888


In [48]:
list_objects = []
json_object1 = ds_1_Transaction.to_dict()
json_object2 = ds_2_Transaction.to_dict()
list_objects.append(json_object1)
list_objects.append(json_object2)
json_str = json.dumps(list_objects, indent=4)
with open("transactions/double_spend.json", "w") as outfile:
    outfile.write(json_str)


Input and Output Match Testfile 

In [49]:
total_sum_output = []
total_sum_input = []
sum_input_1 = transaction_input("1", output_2, None)
sum_output_1 = transaction_output("50", keys_list['pubkey'][0].encode(encoder=Base64Encoder).decode(), None)
sum_output_2 = transaction_output("25", keys_list['pubkey'][7].encode(encoder=Base64Encoder).decode(), None)
total_sum_input.append(sum_input_1)
total_sum_output.append(sum_output_1)
total_sum_output.append(sum_output_2)
sum_sig = createSignature(json.dumps(([i.to_dict() for i in total_sum_input])),json.dumps([o.to_dict() for o in total_sum_output]), keys_list['sk'][1])
sum_1_Transaction = Transaction(total_sum_input, total_sum_output, sum_sig, None, False)

1st hash :3fd3114f69c6f1eacac040066e91e32efa8b74837d605d29ec2d2fe9fe97cb46


Create second transaction 

In [50]:
total_sum_input_1 = []
total_sum_output_1 = []
sum_input_2 = transaction_input("1", output_4, None)
sum_output_3 = transaction_output("30", keys_list['pubkey'][3].encode(encoder=Base64Encoder).decode(), None)
sum_output_4 = transaction_output("40", keys_list['pubkey'][7].encode(encoder=Base64Encoder).decode(), None)
total_sum_input_1.append(sum_input_2)
total_sum_output_1.append(sum_output_3)
total_sum_output_1.append(sum_output_4)
second_sum_sig = createSignature(json.dumps(([i.to_dict() for i in total_sum_input_1])),json.dumps([o.to_dict() for o in total_sum_output_1]), keys_list['sk'][3])
sum_2_Transaction = Transaction(total_sum_input_1, total_sum_output_1, second_sum_sig, None, False)

1st hash :5958a90cfba407237e46347abb5dae7000f5a334f26e3247b6822db7b434215d


In [51]:
objects = []
obj1 = sum_1_Transaction.to_dict()
obj2 = sum_2_Transaction.to_dict()
objects.append(obj1)
objects.append(obj2)
json_str = json.dumps(objects, indent=4)
with open("transactions/InputandOutputSum.json", "w") as outfile:
    outfile.write(json_str)

Transaction Inputs Exist 

In [52]:
ex_input_1_list = []
ex_output_1_list = []
ex_input_1 = transaction_input("1", output_5, None)
ex_output_1 = transaction_output("50", keys_list['pubkey'][4].encode(encoder=Base64Encoder).decode(), None)
ex_output_2 = transaction_output("50", keys_list['pubkey'][5].encode(encoder=Base64Encoder).decode(), None)
ex_input_1_list.append(ex_input_1)
ex_output_1_list.append(ex_output_1)
ex_output_1_list.append(ex_output_2)
ex_sig = createSignature(json.dumps(([i.to_dict() for i in ex_input_1_list])),json.dumps([o.to_dict() for o in ex_output_1_list]), keys_list['sk'][4])
ex_Transaction_1 = Transaction(ex_input_1_list, ex_output_1_list, ex_sig, None, False)

1st hash :ce98c9ef493cf46685df68ca73d74143288f264fdf783a9bba80d55d6c310879


In [53]:
fake_input_1_list = []
fake_output_1_list = []
fake_output = transaction_output("72", keys_list['pubkey'][3].encode(encoder=Base64Encoder).decode(), None)
fake_input = transaction_input("1", fake_output, None)
fake_out_1 = transaction_output("40", keys_list['pubkey'][3].encode(encoder=Base64Encoder).decode(), None)
fake_out_2 = transaction_output("52", keys_list['pubkey'][2].encode(encoder=Base64Encoder).decode(), None)
fake_input_1_list.append(fake_input)
fake_output_1_list.append(fake_out_1)
fake_output_1_list.append(fake_out_2)
fake_sig = createSignature(json.dumps(([i.to_dict() for i in fake_input_1_list])),json.dumps([o.to_dict() for o in fake_output_1_list]), keys_list['sk'][3])
fake_Transaction = Transaction(fake_input_1_list, fake_output_1_list, fake_sig, None, False)

1st hash :44c509754a54bc6d7c49a29cd8452ec77df4b5ec49340df7cb5f3854a9960a92


In [54]:
object1 = ex_Transaction_1.to_dict()
object2 = fake_Transaction.to_dict()
objects = []
objects.append(object1)
objects.append(object2)
json_str = json.dumps(objects, indent=4)
with open("transactions/InputTxNotExist.json", "w") as outfile:
    outfile.write(json_str)

Verified List of Transactions

In [55]:
v_input_1_list = []
v_output_1_list = []
v1_input = transaction_input("1", output_1, None)
v1_output = transaction_output("100", keys_list['pubkey'][1].encode(encoder=Base64Encoder).decode(), None)
v_input_1_list.append(v1_input)
v_output_1_list.append(v1_output)
sig = createSignature(json.dumps(([i.to_dict() for i in v_input_1_list])),json.dumps([o.to_dict() for o in v_output_1_list]), keys_list['sk'][0])
v_transaction1 = Transaction(v_input_1_list, v_output_1_list, sig, None, False)

1st hash :de42d62bceae3a96c411528379d6e28a6f956a2fda6d2c51fe0075838d3df5d0


In [56]:
prev_transaction_number = v_transaction1.tx_number
v_input_2_list = []
v_output_2_list = []
v2_input = transaction_input(prev_transaction_number, v1_output, None)
v2_output = transaction_output("100", keys_list['pubkey'][2].encode(encoder=Base64Encoder).decode(), None)
v_input_2_list.append(v2_input)
v_output_2_list.append(v2_output)
sig2 = createSignature(json.dumps(([i.to_dict() for i in v_input_2_list])),json.dumps([o.to_dict() for o in v_output_2_list]), keys_list['sk'][1])
v_transaction2 = Transaction(v_input_2_list, v_output_2_list, sig2, None, False)

1st hash :38250d8ba21b72bf520ad69721e376cfd71fed7e921628aac0d424f6387a7e57


In [57]:
prev_transaction_number = v_transaction2.tx_number
v_input_3_list = []
v_output_3_list = []
v3_input = transaction_input(prev_transaction_number, v2_output, None)
v3_output = transaction_output("100", keys_list['pubkey'][3].encode(encoder=Base64Encoder).decode(), None)
v_input_3_list.append(v3_input)
v_output_3_list.append(v3_output)
sig3 = createSignature(json.dumps(([i.to_dict() for i in v_input_3_list])),json.dumps([o.to_dict() for o in v_output_3_list]), keys_list['sk'][2])
v_transaction3 = Transaction(v_input_3_list, v_output_3_list, sig3, None, False)

1st hash :9ceae17c26f5e0373245ccab5d541a2b3b109d7e824d961de4833b1219627e72


In [58]:
prev_transaction_number = v_transaction3.tx_number
v_input_4_list = []
v_output_4_list = []
v4_input = transaction_input(prev_transaction_number, v3_output, None)
v4_output = transaction_output("100", keys_list['pubkey'][4].encode(encoder=Base64Encoder).decode(), None)
v_input_4_list.append(v4_input)
v_output_4_list.append(v4_output)
sig4 = createSignature(json.dumps(([i.to_dict() for i in v_input_4_list])),json.dumps([o.to_dict() for o in v_output_4_list]), keys_list['sk'][3])
v_transaction4 = Transaction(v_input_4_list, v_output_4_list, sig4, None, False)

1st hash :87b7ad7518f76b393e89eb4ccccce619870a586872ab6633d978c792b8f59030


In [59]:
prev_transaction_number = v_transaction4.tx_number
v_input_5_list = []
v_output_5_list = []
v5_input = transaction_input(prev_transaction_number, v4_output, None)
v5_output = transaction_output("100", keys_list['pubkey'][5].encode(encoder=Base64Encoder).decode(), None)
v_input_5_list.append(v5_input)
v_output_5_list.append(v5_output)
sig5 = createSignature(json.dumps(([i.to_dict() for i in v_input_5_list])),json.dumps([o.to_dict() for o in v_output_5_list]), keys_list['sk'][4])
v_transaction5 = Transaction(v_input_5_list, v_output_5_list, sig5, None, False)

1st hash :f66ed76c08bfe79998da6f854ab41eadafb6da30d5085e0d9d34c1e57e32e4c5


In [60]:
prev_transaction_number = v_transaction5.tx_number
v_input_6_list = []
v_output_6_list = []
v6_input = transaction_input(prev_transaction_number, v5_output, None)
v6_output = transaction_output("100", keys_list['pubkey'][6].encode(encoder=Base64Encoder).decode(), None)
v_input_6_list.append(v6_input)
v_output_6_list.append(v6_output)
sig6 = createSignature(json.dumps(([i.to_dict() for i in v_input_6_list])),json.dumps([o.to_dict() for o in v_output_6_list]), keys_list['sk'][5])
v_transaction6 = Transaction(v_input_6_list, v_output_6_list, sig6, None, False)

1st hash :0c813777ef93df4cd20769ef66e93c59ba36010ee4d96c12b6f4426ce663ccb2


In [61]:
prev_transaction_number = v_transaction6.tx_number
v_input_7_list = []
v_output_7_list = []
v7_input = transaction_input(prev_transaction_number, v6_output, None)
v7_output = transaction_output("100", keys_list['pubkey'][7].encode(encoder=Base64Encoder).decode(), None)
v_input_7_list.append(v7_input)
v_output_7_list.append(v7_output)
sig7 = createSignature(json.dumps(([i.to_dict() for i in v_input_7_list])),json.dumps([o.to_dict() for o in v_output_7_list]), keys_list['sk'][6])
v_transaction7 = Transaction(v_input_7_list, v_output_7_list, sig7, None, False)

1st hash :42ef7a5c9f5c5a353a6e59151a1c44ddaf27d731ebd7c0955aac1b79c8b6aa15


In [62]:
prev_transaction_number = v_transaction7.tx_number
v_input_8_list = []
v_output_8_list = []
v8_input = transaction_input(prev_transaction_number, v7_output, None)
v8_output = transaction_output("100", keys_list['pubkey'][0].encode(encoder=Base64Encoder).decode(), None)
v_input_8_list.append(v8_input)
v_output_8_list.append(v8_output)
sig8 = createSignature(json.dumps(([i.to_dict() for i in v_input_8_list])),json.dumps([o.to_dict() for o in v_output_8_list]), keys_list['sk'][7])
v_transaction8 = Transaction(v_input_8_list, v_output_8_list, sig8, None, False)

1st hash :7540d51198ddc1980c60e7d36afe2a4d2ea9d6b2e02a02ebd64004d3f9f91f70


In [63]:
prev_transaction_number = v_transaction8.tx_number
v_input_9_list = []
v_output_9_list = []
v9_input = transaction_input(prev_transaction_number, v8_output, None)
v9_output = transaction_output("100", keys_list['pubkey'][1].encode(encoder=Base64Encoder).decode(), None)
v_input_9_list.append(v9_input)
v_output_9_list.append(v9_output)
sig9 = createSignature(json.dumps(([i.to_dict() for i in v_input_9_list])),json.dumps([o.to_dict() for o in v_output_9_list]), keys_list['sk'][0])
v_transaction9 = Transaction(v_input_9_list, v_output_9_list, sig9, None, False)

1st hash :36b1ba160bd2898ab84c4eab2ac6e6ea033009b18cdd137b7e4c67fceef1e9ec


In [64]:
prev_transaction_number = v_transaction9.tx_number
v_input_10_list = []
v_output_10_list = []
v10_input = transaction_input(prev_transaction_number, v9_output, None)
v10_output = transaction_output("100", keys_list['pubkey'][2].encode(encoder=Base64Encoder).decode(), None)
v_input_10_list.append(v10_input)
v_output_10_list.append(v10_output)
sig10 = createSignature(json.dumps(([i.to_dict() for i in v_input_10_list])),json.dumps([o.to_dict() for o in v_output_10_list]), keys_list['sk'][1])
v_transaction10 = Transaction(v_input_10_list, v_output_10_list, sig10, None, False)

1st hash :4762728804ca919e487e45cf97c63276d6d6ddc8a33ce969c048f88d8f8eb3fb


In [65]:
prev_transaction_number = v_transaction10.tx_number
v_input_11_list = []
v_output_11_list = []
v11_input = transaction_input(prev_transaction_number, v10_output, None)
v11_output = transaction_output("100", keys_list['pubkey'][3].encode(encoder=Base64Encoder).decode(), None)
v_input_11_list.append(v11_input)
v_output_11_list.append(v11_output)
sig11 = createSignature(json.dumps(([i.to_dict() for i in v_input_11_list])),json.dumps([o.to_dict() for o in v_output_11_list]), keys_list['sk'][2])
v_transaction11 = Transaction(v_input_11_list, v_output_11_list, sig11, None, False)

1st hash :2589f9865a3d669d97c7cfd9786305264f09ce97be45be97a100820a09870d45


In [66]:
prev_transaction_number = v_transaction11.tx_number
v_input_12_list = []
v_output_12_list = []
v12_input = transaction_input(prev_transaction_number, v11_output, None)
v12_output = transaction_output("100", keys_list['pubkey'][4].encode(encoder=Base64Encoder).decode(), None)
v_input_12_list.append(v12_input)
v_output_12_list.append(v12_output)
sig12 = createSignature(json.dumps(([i.to_dict() for i in v_input_12_list])),json.dumps([o.to_dict() for o in v_output_12_list]), keys_list['sk'][3])
v_transaction12 = Transaction(v_input_12_list, v_output_12_list, sig12, None, False)

1st hash :97bc7f63d13aa763cc5da2e44ba8b0bbb7748300d1a85d2b00718773a936cae5


In [67]:
prev_transaction_number = v_transaction12.tx_number
v_input_13_list = []
v_output_13_list = []
v13_input = transaction_input(prev_transaction_number, v12_output, None)
v13_output = transaction_output("100", keys_list['pubkey'][5].encode(encoder=Base64Encoder).decode(), None)
v_input_13_list.append(v13_input)
v_output_13_list.append(v13_output)
sig13 = createSignature(json.dumps(([i.to_dict() for i in v_input_13_list])),json.dumps([o.to_dict() for o in v_output_13_list]), keys_list['sk'][4])
v_transaction13 = Transaction(v_input_13_list, v_output_13_list, sig13, None, False)

1st hash :02ef597eb9ed3b03dfc92d272680f676a99c864af1a320c8456aa03d0b2167c0


In [68]:
prev_transaction_number = v_transaction13.tx_number
v_input_14_list = []
v_output_14_list = []
v14_input = transaction_input(prev_transaction_number, v13_output, None)
v14_output = transaction_output("100", keys_list['pubkey'][6].encode(encoder=Base64Encoder).decode(), None)
v_input_14_list.append(v14_input)
v_output_14_list.append(v14_output)
sig14 = createSignature(json.dumps(([i.to_dict() for i in v_input_14_list])),json.dumps([o.to_dict() for o in v_output_14_list]), keys_list['sk'][5])
v_transaction14 = Transaction(v_input_14_list, v_output_14_list, sig14, None, False)

1st hash :6f862f451639ac9c36e4533e1fea5a057c887f808b171da91671f1c1bded26df


In [69]:
prev_transaction_number = v_transaction14.tx_number
v_input_15_list = []
v_output_15_list = []
v15_input = transaction_input(prev_transaction_number, v14_output, None)
v15_output = transaction_output("100", keys_list['pubkey'][7].encode(encoder=Base64Encoder).decode(), None)
v_input_15_list.append(v15_input)
v_output_15_list.append(v15_output)
sig15 = createSignature(json.dumps(([i.to_dict() for i in v_input_15_list])),json.dumps([o.to_dict() for o in v_output_15_list]), keys_list['sk'][6])
v_transaction15 = Transaction(v_input_15_list, v_output_15_list, sig15, None, False)

1st hash :1a4eccaa873912a505e3311f673cefb0610dfffa0237c01fa163ab19354f8db4


In [72]:
transactions = []
transaction1 = v_transaction1.to_dict()
transaction2 = v_transaction2.to_dict()
transaction3 = v_transaction3.to_dict()
transaction4 = v_transaction4.to_dict()
transaction5 = v_transaction5.to_dict()
transaction6 = v_transaction6.to_dict()
transaction7 = v_transaction7.to_dict()
transaction8 = v_transaction8.to_dict()
transaction9 = v_transaction9.to_dict()
transaction10 = v_transaction10.to_dict()
transaction11 = v_transaction11.to_dict()
transaction12 = v_transaction12.to_dict()
transaction13 = v_transaction13.to_dict()
transaction14 = v_transaction14.to_dict()
transaction15 = v_transaction15.to_dict()
transactions.append(transaction1)
transactions.append(transaction2)
transactions.append(transaction3)
transactions.append(transaction4)
transactions.append(transaction5)
transactions.append(transaction6)
transactions.append(transaction7)
transactions.append(transaction8)
transactions.append(transaction9)
transactions.append(transaction10)
transactions.append(transaction11)
transactions.append(transaction12)
transactions.append(transaction13)
transactions.append(transaction14)
transactions.append(transaction15)
json_str = json.dumps(transactions, indent=4)
with open("transactions/Verified_Tx.json", "w") as outfile:
    outfile.write(json_str)